In [91]:
import pandas as pd
import json

df = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Nehal\JSON Embedding and Matching\OUTPUTS\user_similarity_ranking.csv")
df['similarity_score'] = df['similarity_score'].apply(lambda x: int(x*100))
df3 = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Mapping\Output\Data_Analyst_scored.csv")

# Open and read the JSON file
with open(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Job_description_features\Data_Analyst_features.json", 'r', encoding='utf-8') as file:
    job_data = json.load(file)

final_df = pd.merge( df3,df[['USER_ID','similarity_score']], how='inner', on='USER_ID')

In [131]:
df.head()

,USER_ID,similarity_score,user_rank
0,2540340,100,1
1,2452725,93,2
2,2920938,89,3
3,2433117,89,4
4,2433142,88,5


In [132]:
df3.head()

,USER_ID,LAST_NAME,Req_SQL_score,Req_Microsoft Power BI_score,Req_Databricks_score,english_score,french_score,experience_score,Pref_DAX_score,Pref_Microsoft Power BI_score,Pref_Matillion ETL_score,availability_score
0,2843838,Ramos,50.0,0.0,0.0,100.0,100.0,10,0.0,0.0,0.0,0.0
1,2479537,Small,0.0,0.0,0.0,90.0,100.0,30,0.0,0.0,0.0,0.0
2,2533337,Evans,50.0,0.0,0.0,50.0,100.0,30,20.0,0.0,0.0,75.0
3,2446382,Pittman,80.0,80.0,0.0,60.0,100.0,30,0.0,80.0,0.0,72.5
4,2433124,Thomas,0.0,0.0,0.0,100.0,100.0,30,0.0,0.0,0.0,75.0


In [133]:
job_data

{'Job Title': 'Data Analyst - Marketing Analytics',
 'Required Skills': [{'skill': 'SQL', 'level': 3},
  {'skill': 'Power BI', 'level': 3},
  {'skill': 'Databricks', 'level': 3},
  {'skill': 'DAX', 'level': 2},
  {'skill': 'Power Query', 'level': 2},
  {'skill': 'ETL', 'level': 2},
  {'skill': 'Data Analysis', 'level': 1},
  {'skill': 'Data Modeling', 'level': 1},
  {'skill': 'Optimization', 'level': 1}],
 'Preferred Skills': [{'skill': 'Marketing Data', 'level': 1}],
 'Experience Required': 5.0,
 'Languages': [{'language': 'English', 'level': 3},
  {'language': 'French', 'level': 3}],
 'Responsibilities': ['Manage and improve marketing datasets.',
  'Develop dashboards and reports in Power BI.',
  'Optimize the performance of Power BI datasets.',
  'Integrate data from various platforms (Google Analytics, CRM, marketing tools).',
  'Write efficient SQL, DAX, and Power Query code.',
  'Collaborate closely with analysts to understand their needs.',
  'Create queries in Databricks and SS

# Method 1

In [75]:
import json
import pandas as pd

def extract_scores(json_data):
    scores = {
        'Req_SQL_score': 0,
        'Req_Microsoft Power BI_score': 0,
        'Req_Databricks_score': 0,
        'english_score': 0,
        'french_score': 0,
        'experience_score': 0,
        'Pref_DAX_score': 0,
        'Pref_Microsoft Power BI_score': 0,
        'Pref_Matillion ETL_score': 0
    }
    
    # Extract required skills
    for skill in json_data.get('Required Skills', []):
        if skill['skill'] == 'SQL':
            scores['Req_SQL_score'] = skill['level']
        elif skill['skill'] == 'Power BI':
            scores['Req_Microsoft Power BI_score'] = skill['level']
        elif skill['skill'] == 'Databricks':
            scores['Req_Databricks_score'] = skill['level']
        elif skill['skill'] == 'DAX':
            scores['Pref_DAX_score'] = skill['level']
        elif skill['skill'] == 'ETL':
            scores['Pref_Matillion ETL_score'] = skill['level']
    
    # Extract language skills
    for language in json_data.get('Languages', []):
        if language['language'] == 'English':
            scores['english_score'] = language['level']
        elif language['language'] == 'French':
            scores['french_score'] = language['level']
    
    # Extract experience
    scores['experience_score'] = json_data.get('Experience Required', 0)
    
    return scores


weights = extract_scores(job_data)

In [76]:
weights = extract_scores(job_data)  # This gives the weights (levels)

# Multiply each candidate's scores by the weights
weighted_scores = final_df * pd.Series(weights)

# Calculate the average weighted score per candidate
weighted_avg = weighted_scores.sum(axis=1) / sum(weights.values())

print("Weighted Average Scores:")
print(weighted_avg)

Weighted Average Scores:
0      33.333333
1           30.0
2      32.916667
3          46.25
4          31.25
         ...    
114        28.75
115        43.75
116        28.75
117    43.333333
118    12.083333
Length: 119, dtype: object


In [77]:
# Add weighted average column
final_df['Weighted_Avg'] = None  # Initialize column
final_df.loc[weighted_scores.index, 'Weighted_Avg'] = weighted_scores.mean(axis=1)

# Add weighted average column
final_df['Weighted_Avg'] = None  # Initialize column
final_df.loc[weighted_scores.index, 'Weighted_Avg'] = weighted_avg

final_df

,USER_ID,LAST_NAME,Req_SQL_score,Req_Microsoft Power BI_score,Req_Databricks_score,english_score,french_score,experience_score,Pref_DAX_score,Pref_Microsoft Power BI_score,Pref_Matillion ETL_score,availability_score,similarity_score,Weighted_Avg
0,2843838,Ramos,50.0,0.0,0.0,100.0,100.0,10,0.0,0.0,0.0,0.0,33,33.333333
1,2479537,Small,0.0,0.0,0.0,90.0,100.0,30,0.0,0.0,0.0,0.0,33,30.0
2,2533337,Evans,50.0,0.0,0.0,50.0,100.0,30,20.0,0.0,0.0,75.0,53,32.916667
3,2446382,Pittman,80.0,80.0,0.0,60.0,100.0,30,0.0,80.0,0.0,72.5,45,46.25
4,2433124,Thomas,0.0,0.0,0.0,100.0,100.0,30,0.0,0.0,0.0,75.0,64,31.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,0.0,0.0,0.0,100.0,80.0,30,0.0,0.0,0.0,75.0,55,28.75
115,2456947,Riddle,100.0,0.0,0.0,100.0,100.0,30,0.0,0.0,0.0,100.0,62,43.75
116,2447672,Gonzalez,0.0,0.0,0.0,80.0,100.0,30,0.0,0.0,0.0,97.5,50,28.75
117,2789547,Williams,80.0,80.0,0.0,70.0,100.0,10,0.0,80.0,0.0,72.5,79,43.333333


In [78]:
final_df[final_df['availability_score']!=0].sort_values(by='Weighted_Avg', ascending=False)

,USER_ID,LAST_NAME,Req_SQL_score,Req_Microsoft Power BI_score,Req_Databricks_score,english_score,french_score,experience_score,Pref_DAX_score,Pref_Microsoft Power BI_score,Pref_Matillion ETL_score,availability_score,similarity_score,Weighted_Avg
85,2676328,Green,100.0,90.0,0.0,90.0,100.0,30,90.0,90.0,0.0,50.0,64,61.25
90,2501974,Mcgee,80.0,100.0,0.0,100.0,100.0,30,80.0,100.0,0.0,50.0,47,60.416667
79,2433111,Richards,100.0,80.0,0.0,100.0,100.0,30,70.0,80.0,0.0,50.0,85,59.583333
54,2433109,Haley,100.0,0.0,0.0,80.0,100.0,30,100.0,0.0,100.0,50.0,62,57.916667
87,2551856,Hines,80.0,100.0,0.0,80.0,100.0,30,80.0,100.0,0.0,75.0,83,57.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2799951,Soto,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,75.0,0,2.083333
47,2967233,Ballard,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,95.0,0,2.083333
25,2977311,Moore,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,97.5,55,2.083333
6,2936770,Choi,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,72.5,76,2.083333


# Method - 2

In [79]:
import json

def extract_scores(json_data):
    # Initialize a dictionary with all possible scores set to 0 (or another default)
    scores = {
        'Req_SQL_score': 0,
        'Req_Microsoft Power BI_score': 0,
        'Req_Databricks_score': 0,
        'english_score': 0,
        'french_score': 0,
        'Pref_DAX_score': 0,
        'Pref_Microsoft Power BI_score': 0,
        'Pref_Matillion ETL_score': 0
    }
    
    # Extract required skills
    for skill in json_data.get('Required Skills', []):
        if skill['skill'] == 'SQL':
            scores['Req_SQL_score'] = skill['level']
        elif skill['skill'] == 'Power BI':
            scores['Req_Microsoft Power BI_score'] = skill['level']
        elif skill['skill'] == 'Databricks':
            scores['Req_Databricks_score'] = skill['level']
        elif skill['skill'] == 'DAX':
            scores['Pref_DAX_score'] = skill['level']
        elif skill['skill'] == 'ETL':
            scores['Pref_Matillion ETL_score'] = skill['level']
    
    # Extract language skills
    for language in json_data.get('Languages', []):
        if language['language'] == 'English':
            scores['english_score'] = language['level']
        elif language['language'] == 'French':
            scores['french_score'] = language['level']
    
    return scores

result = extract_scores(job_data)
print(result)

{'Req_SQL_score': 3, 'Req_Microsoft Power BI_score': 3, 'Req_Databricks_score': 3, 'english_score': 3, 'french_score': 3, 'Pref_DAX_score': 2, 'Pref_Microsoft Power BI_score': 0, 'Pref_Matillion ETL_score': 2}


In [90]:
final_df.drop(columns=['english_score',	'french_score','experience_score'],axis=1) * pd.Series(weights)

,LAST_NAME,Pref_DAX_score,Pref_Matillion ETL_score,Pref_Microsoft Power BI_score,Req_Databricks_score,Req_Microsoft Power BI_score,Req_SQL_score,USER_ID,Weighted_Avg,availability_score,english_score,french_score,similarity_score
24,NaN,0.0,0.0,0.0,300.0,300.0,300.0,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,200.0,0.0,0.0,0.0,300.0,300.0,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,160.0,0.0,0.0,0.0,300.0,300.0,NaN,NaN,NaN,NaN,NaN,NaN
104,NaN,120.0,0.0,0.0,0.0,300.0,300.0,NaN,NaN,NaN,NaN,NaN,NaN
85,NaN,180.0,0.0,0.0,0.0,270.0,300.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
37,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
77,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
weights = extract_scores(job_data)  # This gives the weights (levels)

# Multiply each candidate's scores by the weights
weighted_scores = final_df.drop(columns=['english_score',	'french_score','experience_score'],axis=1) * pd.Series(weights)

# Calculate the average weighted score per candidate
weighted_avg = weighted_scores.sum(axis=1) / sum(weights.values())

print("Weighted Average Scores:")
print(weighted_avg)

Weighted Average Scores:
0      39.473684
1           30.0
2      33.684211
3      50.526316
4      31.578947
         ...    
114    28.421053
115    47.368421
116    28.421053
117    52.105263
118    12.631579
Length: 119, dtype: object


In [ ]:
final_df.drop(columns=['english_score',	'french_score','experience_score'],axis=1).loc[weighted_scores.index, 'Weighted_Avg']

,USER_ID,LAST_NAME,Req_SQL_score,Req_Microsoft Power BI_score,Req_Databricks_score,Pref_DAX_score,Pref_Microsoft Power BI_score,Pref_Matillion ETL_score,availability_score,similarity_score,Weighted_Avg
24,2452725,Harris,100.0,100.0,100.0,0.0,100.0,0.0,0.0,93,None
19,2445183,Valencia,100.0,100.0,0.0,100.0,100.0,0.0,0.0,58,None
51,2453233,Boyd,100.0,100.0,0.0,80.0,100.0,0.0,0.0,43,None
104,2457327,Diaz,100.0,100.0,0.0,60.0,100.0,0.0,0.0,52,None
85,2676328,Green,100.0,90.0,0.0,90.0,90.0,0.0,50.0,64,None
...,...,...,...,...,...,...,...,...,...,...,...
43,2903451,Huynh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59,None
37,2726335,Thompson,0.0,0.0,0.0,0.0,0.0,0.0,77.5,76,None
70,2664664,Bowman,0.0,0.0,0.0,0.0,0.0,0.0,100.0,46,None
77,2708468,Johnson,0.0,0.0,0.0,0.0,0.0,0.0,75.0,73,None


In [86]:
# Add weighted average column
final_df['Weighted_Avg'] = None  # Initialize column
final_df.drop(columns=['english_score',	'french_score','experience_score'],axis=1).loc[weighted_scores.index, 'Weighted_Avg'] = weighted_avg
final_df

C:\Users\hp\AppData\Local\Temp\ipykernel_10092\3354201211.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  final_df.drop(columns=['english_score',	'french_score','experience_score'],axis=1).loc[weighted_scores.index, 'Weighted_Avg'] = wei

,USER_ID,LAST_NAME,Req_SQL_score,Req_Microsoft Power BI_score,Req_Databricks_score,english_score,french_score,experience_score,Pref_DAX_score,Pref_Microsoft Power BI_score,Pref_Matillion ETL_score,availability_score,similarity_score,Weighted_Avg
24,2452725,Harris,100.0,100.0,100.0,100.0,100.0,30,0.0,100.0,0.0,0.0,93,None
19,2445183,Valencia,100.0,100.0,0.0,100.0,100.0,30,100.0,100.0,0.0,0.0,58,None
51,2453233,Boyd,100.0,100.0,0.0,100.0,100.0,30,80.0,100.0,0.0,0.0,43,None
104,2457327,Diaz,100.0,100.0,0.0,100.0,100.0,30,60.0,100.0,0.0,0.0,52,None
85,2676328,Green,100.0,90.0,0.0,90.0,100.0,30,90.0,90.0,0.0,50.0,64,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,2903451,Huynh,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,0.0,59,None
37,2726335,Thompson,0.0,0.0,0.0,0.0,0.0,30,0.0,0.0,0.0,77.5,76,None
70,2664664,Bowman,0.0,0.0,0.0,0.0,0.0,30,0.0,0.0,0.0,100.0,46,None
77,2708468,Johnson,0.0,0.0,0.0,0.0,0.0,30,0.0,0.0,0.0,75.0,73,None


In [82]:
final_df.sort_values(by='Weighted_Avg', ascending=False, inplace=True)

In [ ]:
final_df['experience_score']* category_weights['Experience'] + final_df['Weighted_Avg'] * category_weights['Skills'] + final_df['Languages'] * category_weights['Languages'] + final_df['similarity_score'] * category_weights['Similarity']

,USER_ID,LAST_NAME,Req_SQL_score,Req_Microsoft Power BI_score,Req_Databricks_score,english_score,french_score,experience_score,Pref_DAX_score,Pref_Microsoft Power BI_score,Pref_Matillion ETL_score,availability_score,similarity_score,Weighted_Avg
24,2452725,Harris,100.0,100.0,100.0,100.0,100.0,30,0.0,100.0,0.0,0.0,93,78.947368
19,2445183,Valencia,100.0,100.0,0.0,100.0,100.0,30,100.0,100.0,0.0,0.0,58,73.684211
51,2453233,Boyd,100.0,100.0,0.0,100.0,100.0,30,80.0,100.0,0.0,0.0,43,71.578947
104,2457327,Diaz,100.0,100.0,0.0,100.0,100.0,30,60.0,100.0,0.0,0.0,52,69.473684
85,2676328,Green,100.0,90.0,0.0,90.0,100.0,30,90.0,90.0,0.0,50.0,64,69.473684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,2903451,Huynh,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,0.0,59,0.0
37,2726335,Thompson,0.0,0.0,0.0,0.0,0.0,30,0.0,0.0,0.0,77.5,76,0.0
70,2664664,Bowman,0.0,0.0,0.0,0.0,0.0,30,0.0,0.0,0.0,100.0,46,0.0
77,2708468,Johnson,0.0,0.0,0.0,0.0,0.0,30,0.0,0.0,0.0,75.0,73,0.0


In [84]:
category_weights = {
    "Skills": 0.4,   # 60% weight
    "Languages": 0.1,          # 20% weight
    "Experience": 0.2,          # 20% weight
    "Similarity": 0.3          # 20% weight
}

In [42]:
column_categories = {
    "Req_SQL_score": "Technical Skills",
    "Req_Microsoft Power BI_score": "Technical Skills",
    "Req_Databricks_score": "Technical Skills",
    "Pref_DAX_score": "Technical Skills",
    "Pref_Matillion ETL_score": "Technical Skills",
    "english_score": "Languages",
    "french_score": "Languages",
    "experience_score": "Experience",
}

In [44]:
weights = extract_scores(job_data)

# Multiply each candidate's scores by the weights
weighted_scores = final_df * pd.Series(weights)

# Calculate category-wise weighted averages
category_scores = {}
for category, weight in category_weights.items():
    # Get columns in this category
    cols = [col for col, cat in column_categories.items() if cat == category]
    # Compute weighted average for the category
    category_scores[f"{category}_Score"] = weighted_scores[cols].sum(axis=1) / sum(weights[col] for col in cols) * weight

# Combine into a DataFrame
category_df = pd.DataFrame(category_scores)

# Compute final weighted score
category_df["Total_Weighted_Score"] = category_df.sum(axis=1)

# Add to original DataFrame
df_final = pd.concat([df, category_df], axis=1)

df_final

,USER_ID,similarity_score,user_rank,Technical Skills_Score,Languages_Score,Experience_Score,Total_Weighted_Score
0,2540340,1.000000,1,6.923077,20.0,2.0,28.923077
1,2452725,0.936590,2,0.000000,19.0,6.0,25.000000
2,2920938,0.899009,3,8.769231,15.0,6.0,29.769231
3,2433117,0.895343,4,22.153846,16.0,6.0,44.153846
4,2433142,0.886154,5,0.000000,20.0,6.0,26.000000
...,...,...,...,...,...,...,...
114,2471586,0.000000,115,0.000000,18.0,6.0,24.000000
115,2967233,0.000000,116,13.846154,20.0,6.0,39.846154
116,2799951,0.000000,117,0.000000,18.0,6.0,24.000000
117,2448418,0.000000,118,22.153846,17.0,2.0,41.153846


In [45]:
df_final.sort_values(by='Total_Weighted_Score', ascending=False)

,USER_ID,similarity_score,user_rank,Technical Skills_Score,Languages_Score,Experience_Score,Total_Weighted_Score
24,2433122,0.728897,25,41.538462,20.0,6.0,67.538462
19,2936770,0.760063,20,36.923077,20.0,6.0,62.923077
51,2445183,0.580266,52,35.076923,20.0,6.0,61.076923
85,2437177,0.460675,86,34.615385,19.0,6.0,59.615385
104,2433144,0.378000,105,33.230769,20.0,6.0,59.230769
...,...,...,...,...,...,...,...
6,2433134,0.875624,7,0.000000,0.0,2.0,2.000000
47,2433114,0.617364,48,0.000000,0.0,2.0,2.000000
43,2502347,0.637860,44,0.000000,0.0,2.0,2.000000
83,2443208,0.462108,84,0.000000,0.0,2.0,2.000000


# Method 3

In [94]:
import pandas as pd
import json

# # 1. Load your JSON data
# with open('job_data.json', 'r') as file:
#     job_data = json.load(file)

# 2. Define column name mappings between JSON and DataFrame
column_mapping = {
    'SQL': 'Req_SQL_score',
    'Power BI': 'Req_Microsoft Power BI_score',
    'Databricks': 'Req_Databricks_score',
    'DAX': 'Pref_DAX_score',
    'Power Query': None,  # Not in your DataFrame
    'ETL': 'Pref_Matillion ETL_score',
    'Data Analysis': None,  # Not in your DataFrame
    'Data Modeling': None,  # Not in your DataFrame
    'Optimization': None,  # Not in your DataFrame
    'Marketing Data': None,  # Not in your DataFrame
    'English': 'english_score',
    'French': 'french_score'
}

# 3. Extract weights from JSON and map to DataFrame columns
def get_weights(job_data, column_mapping):
    weights = {}
    
    # Process Required Skills
    for skill in job_data['Required Skills']:
        df_col = column_mapping.get(skill['skill'])
        if df_col:
            weights[df_col] = skill['level']
    
    # Process Preferred Skills
    for skill in job_data['Preferred Skills']:
        df_col = column_mapping.get(skill['skill'])
        if df_col:
            weights[df_col] = skill['level']
    
    # Process Languages
    for lang in job_data['Languages']:
        df_col = column_mapping.get(lang['language'])
        if df_col:
            weights[df_col] = lang['level']
    
    return weights

weights = get_weights(job_data, column_mapping)

# 4. Calculate weighted averages
def calculate_weighted_avg(df, weights):
    # Get all available skill columns (excluding languages and experience)
    skill_cols = [col for col in weights.keys() 
                 if col not in ['english_score', 'french_score', 'experience_score']]
    
    # Get language columns
    lang_cols = [col for col in weights.keys() 
                if col in ['english_score', 'french_score']]
    
    # Calculate weighted averages (only for columns that exist)
    df['weighted_avg_skills'] = (
        df[skill_cols].mul([weights[col] for col in skill_cols]).sum(axis=1) / 
        sum(weights[col] for col in skill_cols)
    )
    
    df['weighted_avg_lang'] = (
        df[lang_cols].mul([weights[col] for col in lang_cols]).sum(axis=1) / 
        sum(weights[col] for col in lang_cols)
    )
    
    return df

# 5. Apply to DataFrame
final_df_ = calculate_weighted_avg(final_df, weights)
print(final_df_[['weighted_avg_skills', 'weighted_avg_lang']])

     weighted_avg_skills  weighted_avg_lang
0              11.538462              100.0
1               0.000000               95.0
2              14.615385               75.0
3              36.923077               80.0
4               0.000000              100.0
..                   ...                ...
114             0.000000               90.0
115            23.076923              100.0
116             0.000000               90.0
117            36.923077               85.0
118             0.000000               40.0

[119 rows x 2 columns]


In [114]:
# 1. Define category weights (as per your requirements)
category_weights = {
    "Skills": 0.4,      # 40% weight
    "Languages": 0.1,   # 10% weight
    "Experience": 0.3,  # 20% weight
    "Similarity": 0.2   # 30% weight
}

# 2. Calculate final weighted score
def calculate_final_score(df, category_weights):
    """
    Calculates final weighted score based on category weights
    """    
    # Calculate weighted components
    df['weighted_skills'] = df['weighted_avg_skills'] * category_weights['Skills']
    df['weighted_lang'] = df['weighted_avg_lang'] * category_weights['Languages']
    df['weighted_exp'] = df['experience_score'] * category_weights['Experience']
    df['weighted_similarity'] = df['similarity_score'] * category_weights['Similarity']
    
    # Calculate final score (sum of all weighted components)
    df['final_weighted_score'] = (df['weighted_skills'] + 
                                 df['weighted_lang'] + 
                                 df['weighted_exp'] + 
                                 df['weighted_similarity'])
    
    
    return df

# 3. Apply the calculation
final_df = calculate_final_score(final_df_, category_weights)

# Show important columns
result_columns = [
    'availability_score',
    'similarity_score',
    'weighted_avg_skills',
    'weighted_avg_lang',
    'experience_score',
    'final_weighted_score',
    'final_score_percentage'
]
final_df[final_df['availability_score']!=0].sort_values(by='final_weighted_score', ascending=False)

,USER_ID,LAST_NAME,Req_SQL_score,Req_Microsoft Power BI_score,Req_Databricks_score,english_score,french_score,experience_score,Pref_DAX_score,Pref_Microsoft Power BI_score,Pref_Matillion ETL_score,availability_score,similarity_score,weighted_avg_skills,weighted_avg_lang,weighted_skills,weighted_lang,weighted_exp,weighted_similarity,final_weighted_score,final_score_percentage
79,2433111,Richards,100.0,80.0,0.0,100.0,100.0,30,70.0,80.0,0.0,50.0,85,52.307692,100.0,20.923077,10.0,9.0,17.0,56.923077,6242.307692
87,2551856,Hines,80.0,100.0,0.0,80.0,100.0,30,80.0,100.0,0.0,75.0,83,53.846154,90.0,21.538462,9.0,9.0,16.6,56.138462,6143.846154
73,2445149,Austin,100.0,80.0,0.0,80.0,100.0,30,80.0,80.0,0.0,77.5,79,53.846154,90.0,21.538462,9.0,9.0,15.8,55.338462,6023.846154
85,2676328,Green,100.0,90.0,0.0,90.0,100.0,30,90.0,90.0,0.0,50.0,64,57.692308,95.0,23.076923,9.5,9.0,12.8,54.376923,5777.692308
96,2433112,Dixon,100.0,40.0,0.0,80.0,100.0,30,0.0,40.0,80.0,97.5,87,44.615385,90.0,17.846154,9.0,9.0,17.4,53.246154,5894.615385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,2977311,Moore,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,97.5,55,0.000000,0.0,0.000000,0.0,3.0,11.0,14.000000,1850.000000
41,2471586,Owens,0.0,0.0,0.0,0.0,100.0,30,0.0,0.0,0.0,100.0,0,0.000000,50.0,0.000000,5.0,9.0,0.0,14.000000,1100.000000
83,2976032,Kelly,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,100.0,16,0.000000,0.0,0.000000,0.0,3.0,3.2,6.200000,680.000000
47,2967233,Ballard,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,95.0,0,0.000000,0.0,0.000000,0.0,3.0,0.0,3.000000,200.000000


In [115]:
final_df[final_df['availability_score']!=0].sort_values(by='final_weighted_score', ascending=False).head(10)

,USER_ID,LAST_NAME,Req_SQL_score,Req_Microsoft Power BI_score,Req_Databricks_score,english_score,french_score,experience_score,Pref_DAX_score,Pref_Microsoft Power BI_score,Pref_Matillion ETL_score,availability_score,similarity_score,weighted_avg_skills,weighted_avg_lang,weighted_skills,weighted_lang,weighted_exp,weighted_similarity,final_weighted_score,final_score_percentage
79,2433111,Richards,100.0,80.0,0.0,100.0,100.0,30,70.0,80.0,0.0,50.0,85,52.307692,100.0,20.923077,10.0,9.0,17.0,56.923077,6242.307692
87,2551856,Hines,80.0,100.0,0.0,80.0,100.0,30,80.0,100.0,0.0,75.0,83,53.846154,90.0,21.538462,9.0,9.0,16.6,56.138462,6143.846154
73,2445149,Austin,100.0,80.0,0.0,80.0,100.0,30,80.0,80.0,0.0,77.5,79,53.846154,90.0,21.538462,9.0,9.0,15.8,55.338462,6023.846154
85,2676328,Green,100.0,90.0,0.0,90.0,100.0,30,90.0,90.0,0.0,50.0,64,57.692308,95.0,23.076923,9.5,9.0,12.8,54.376923,5777.692308
96,2433112,Dixon,100.0,40.0,0.0,80.0,100.0,30,0.0,40.0,80.0,97.5,87,44.615385,90.0,17.846154,9.0,9.0,17.4,53.246154,5894.615385
14,2447746,Payne,70.0,100.0,0.0,70.0,100.0,30,70.0,100.0,0.0,100.0,78,50.000000,85.0,20.000000,8.5,9.0,15.6,53.100000,5790.000000
54,2433109,Haley,100.0,0.0,0.0,80.0,100.0,30,100.0,0.0,100.0,50.0,62,53.846154,90.0,21.538462,9.0,9.0,12.4,51.938462,5513.846154
91,2433136,Nielsen,100.0,100.0,0.0,100.0,100.0,30,0.0,100.0,0.0,75.0,71,46.153846,100.0,18.461538,10.0,9.0,14.2,51.661538,5576.153846
42,2578963,Kelly,80.0,90.0,0.0,80.0,100.0,30,80.0,90.0,0.0,75.0,64,51.538462,90.0,20.615385,9.0,9.0,12.8,51.415385,5481.538462
76,2540340,Reeves,50.0,60.0,0.0,70.0,100.0,30,60.0,60.0,0.0,100.0,100,34.615385,85.0,13.846154,8.5,9.0,20.0,51.346154,5834.615385


In [107]:
pd.set_option('display.max_columns', None)  # Show all columns

## Data Engineer

In [ ]:
import pandas as pd
import json

# Load candidate similarity data
similarity_scores_df = pd.read_csv(
    r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Nehal\JSON Embedding and Matching\OUTPUTS\user_similarity_ranking.csv"
)

# Convert similarity scores to percentage (0-100)
similarity_scores_df['similarity_score'] = similarity_scores_df['similarity_score'].apply(lambda x: int(x * 100))

# Load candidate features data
candidate_features_df = pd.read_csv(
    r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Mapping\Output\Data_Engineer_scored.csv"
)

# Load job description data
with open(
    r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Job_description_features\Data_Engineer_features.json",
    'r',
    encoding='utf-8'
) as file:
    job_description_data = json.load(file)

# Merge datasets on USER_ID
merged_candidates_df = pd.merge(
    candidate_features_df,
    similarity_scores_df[['USER_ID', 'similarity_score']],
    how='inner',
    on='USER_ID'
)


In [122]:
import pandas as pd

# Assuming merged_candidates_df contains all these columns:
# Req_Spark_score, Req_Python_score, Req_Scala_score, 
# Req_Microsoft Azure Data Factory_score, Req_Databricks_score,
# french_score, english_score, experience_score, 
# availability_score, similarity_score

# 1. Define category weights
category_weights = {
    "Technical_Skills": 0.4,   # 40% weight
    "Languages": 0.1,          # 10% weight
    "Experience": 0.2,         # 20% weight
    "Availability": 0.0,       # 0% weight (if not used)
    "Similarity": 0.3          # 30% weight
}

# 2. Define which columns belong to which category
column_categories = {
    # Technical Skills
    'Req_Spark_score': 'Technical_Skills',
    'Req_Python_score': 'Technical_Skills',
    'Req_Scala_score': 'Technical_Skills',
    'Req_Microsoft Azure Data Factory_score': 'Technical_Skills',
    'Req_Databricks_score': 'Technical_Skills',
    
    # Languages
    'french_score': 'Languages',
    'english_score': 'Languages',
    
    # Other categories
    'experience_score': 'Experience',
    'availability_score': 'Availability',
    'similarity_score': 'Similarity'
}

# 3. Calculate weighted scores
def calculate_weighted_scores(df, category_weights, column_categories):
    # Initialize all weighted columns
    for category in category_weights:
        df[f'weighted_{category}'] = 0.0
    
    # Calculate weighted scores for each category
    for column, category in column_categories.items():
        if column in df.columns and category in category_weights:
            weight = category_weights[category]
            
            # For technical skills and languages, we'll normalize later
            if category in ['Technical_Skills', 'Languages']:
                df[f'weighted_{category}'] += df[column] * weight
            else:
                # For experience, availability, similarity - direct weighting
                if category == 'Experience':
                    # Normalize experience (assuming 0-5 scale)
                    df[f'weighted_{category}'] = (df[column] / 5.0) * weight
                else:
                    df[f'weighted_{category}'] = df[column] * weight
    
    # Normalize technical skills and languages (divide by number of items)
    tech_skill_cols = [col for col, cat in column_categories.items() 
                      if cat == 'Technical_Skills' and col in df.columns]
    lang_cols = [col for col, cat in column_categories.items() 
                if cat == 'Languages' and col in df.columns]
    
    if tech_skill_cols:
        df['weighted_Technical_Skills'] = df['weighted_Technical_Skills'] / len(tech_skill_cols)
    if lang_cols:
        df['weighted_Languages'] = df['weighted_Languages'] / len(lang_cols)
    
    # Calculate final score
    df['final_weighted_score'] = (df['weighted_Technical_Skills'] +
                                 df['weighted_Languages'] +
                                 df['weighted_Experience'] +
                                 df['weighted_Similarity'])
    
    # Convert to percentage (0-100 scale)
    df['final_score_percentage'] = df['final_weighted_score'] * 100
    
    return df

# 4. Apply the calculation
final_scored_df = calculate_weighted_scores(merged_candidates_df, category_weights, column_categories)

# Show important columns
result_columns = [
    'USER_ID',
    'Req_Spark_score',
    'Req_Python_score',
    'Req_Databricks_score',
    'english_score',
    'french_score',
    'experience_score',
    'similarity_score',
    'final_weighted_score',
    'final_score_percentage'
]

# Sort by final score (descending)
final_results = final_scored_df.sort_values('final_score_percentage', ascending=False)
final_results.head()

,USER_ID,LAST_NAME,Req_Spark_score,Req_Python_score,Req_Scala_score,Req_Microsoft Azure Data Factory_score,Req_Databricks_score,french_score,english_score,experience_score,availability_score,similarity_score,weighted_Technical_Skills,weighted_Languages,weighted_Experience,weighted_Availability,weighted_Similarity,final_weighted_score,final_score_percentage
24,2452725,Harris,0.0,100.0,100.0,0.0,100.0,100.0,100.0,30,100,93,24.0,10.0,1.2,0.0,27.9,63.1,6310.0
55,2433142,Sullivan,0.0,100.0,0.0,80.0,0.0,100.0,100.0,30,100,88,14.4,10.0,1.2,0.0,26.4,52.0,5200.0
117,2789547,Williams,0.0,70.0,30.0,80.0,0.0,100.0,70.0,10,100,79,14.4,8.5,0.4,0.0,23.7,47.0,4700.0
94,2433138,Nelson,0.0,30.0,0.0,100.0,0.0,100.0,80.0,30,100,78,10.4,9.0,1.2,0.0,23.4,44.0,4400.0
87,2551856,Hines,0.0,70.0,0.0,40.0,0.0,100.0,80.0,30,100,83,8.8,9.0,1.2,0.0,24.9,43.9,4390.0


In [124]:
import pandas as pd
import json

# Load candidate-job similarity data
candidate_job_similarity_df = pd.read_csv(
    r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Nehal\JSON Embedding and Matching\OUTPUTS\user_similarity_ranking.csv"
)

# Normalize similarity scores to percentage scale (0-100)
candidate_job_similarity_df['similarity_percentage'] = candidate_job_similarity_df['similarity_score'].apply(lambda x: int(x * 100))

# Load candidate technical assessment data
candidate_technical_scores_df = pd.read_csv(
    r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Mapping\Output\Scrum_Master_scored.csv"
)

# Load job requirements data
with open(
    r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Job_description_features\Scrum_Master_features.json",
    'r',
    encoding='utf-8'
) as file:
    job_requirements_data = json.load(file)

# Merge candidate technical scores with similarity scores
candidate_full_assessment_df = pd.merge(
    candidate_technical_scores_df,
    candidate_job_similarity_df[['USER_ID', 'similarity_percentage']],
    how='inner',
    on='USER_ID'
)

In [128]:
import pandas as pd

# Define category weights (adjust these based on your priorities)
category_weights = {
    "Languages": 0.2,          # 20% weight (French + English)
    "Experience": 0.3,         # 30% weight
    "Availability": 0.1,       # 10% weight
    "Similarity": 0.4          # 40% weight
}

def calculate_final_score(df, weights):
    """
    Calculate weighted final score from available features
    """
    # 1. Normalize scores to comparable scales (0-1)
    df['norm_french'] = df['french_score'] / 5  # Assuming 5-point scale
    df['norm_english'] = df['english_score'] / 5
    df['norm_experience'] = df['experience_score'] / 10  # Assuming 10 years max
    df['norm_availability'] = df['availability_score'] / 100  # Assuming percentage
    
    # 2. Calculate category scores
    df['language_score'] = (df['norm_french'] + df['norm_english']) / 2 * weights['Languages']
    df['experience_score_weighted'] = df['norm_experience'] * weights['Experience']
    df['availability_score_weighted'] = df['norm_availability'] * weights['Availability']
    df['similarity_score_weighted'] = (df['similarity_percentage'] / 100) * weights['Similarity']
    
    # 3. Calculate final composite score (0-1 scale)
    df['composite_score'] = (df['language_score'] + 
                            df['experience_score_weighted'] + 
                            df['availability_score_weighted'] + 
                            df['similarity_score_weighted'])
    
    # 4. Convert to percentage (0-100 scale)
    df['final_score_percentage'] = df['composite_score'] * 100
    
    return df

# Apply scoring function
scored_candidates_df = calculate_final_score(candidate_full_assessment_df, category_weights)

# Select and sort results
result_columns = [
    'USER_ID',
    'french_score',
    'english_score',
    'experience_score',
    'availability_score',
    'similarity_percentage',
    'final_score_percentage'
]

final_results = scored_candidates_df.sort_values(
    'final_score_percentage', 
    ascending=False
)

final_results.head()

,USER_ID,LAST_NAME,french_score,english_score,experience_score,availability_score,similarity_percentage,norm_french,norm_english,norm_experience,norm_availability,language_score,experience_score_weighted,availability_score_weighted,similarity_score_weighted,composite_score,final_score_percentage
24,2452725,Harris,100.0,100.0,50,100,93,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.372,5.972,597.2
16,2433117,Black,100.0,100.0,50,100,89,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.356,5.956,595.6
55,2433142,Sullivan,100.0,100.0,50,100,88,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.352,5.952,595.2
79,2433111,Richards,100.0,100.0,50,100,85,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.340,5.940,594.0
13,2433130,Simpson,100.0,100.0,50,100,76,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.304,5.904,590.4


In [130]:
final_results.head(10)

,USER_ID,LAST_NAME,french_score,english_score,experience_score,availability_score,similarity_percentage,norm_french,norm_english,norm_experience,norm_availability,language_score,experience_score_weighted,availability_score_weighted,similarity_score_weighted,composite_score,final_score_percentage
24,2452725,Harris,100.0,100.0,50,100,93,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.372,5.972,597.2
16,2433117,Black,100.0,100.0,50,100,89,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.356,5.956,595.6
55,2433142,Sullivan,100.0,100.0,50,100,88,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.352,5.952,595.2
79,2433111,Richards,100.0,100.0,50,100,85,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.340,5.940,594.0
13,2433130,Simpson,100.0,100.0,50,100,76,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.304,5.904,590.4
7,2501258,Martinez,100.0,100.0,50,100,74,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.296,5.896,589.6
74,2451804,Hall,100.0,100.0,50,100,72,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.288,5.888,588.8
80,2444211,Lee,100.0,100.0,50,100,72,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.288,5.888,588.8
91,2433136,Nielsen,100.0,100.0,50,100,71,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.284,5.884,588.4
63,2433118,Morrison,100.0,100.0,50,100,67,20.0,20.0,5.0,1.0,4.0,1.5,0.1,0.268,5.868,586.8


# Final code

In [229]:
base_df = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Pivot Processing\Output\Users_Lang_Skills.csv")

In [230]:
base_df.columns

Index(['USER_ID', 'LAST_NAME', 'FIRST_NAME', 'ANNEES_XP', 'Airbyte_level',
       'Alteryx Analytics Hub_level', 'Alteryx Designer_level',
       'Amazon AWS CloudFormation_level', 'Amazon AWS Glue_level',
       'Amazon QuickSight_level',
       ...
       'Teradata Vantage_level', 'Terraform_level', 'VBA_level',
       'Vena Close_level',
       'Vena Solutions Budgeting, Planning & Forecasting_level',
       'Workday Adaptive Planning_level', 'Workiva_level',
       'insightsoftware Longview Planning_level', 'English_level',
       'French_level'],
      dtype='object', length=139)

In [231]:
exclude_columns = ['USER_ID','LAST_NAME','FIRST_NAME','ANNEES_XP','English_level','French_level']

In [232]:
# Apply transformation to all columns except those in exclude_columns
base_df.columns = [
    col.replace('_level', '').lower() if col not in exclude_columns else col
    for col in base_df.columns
]

In [233]:
base_df.head()

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,airbyte,alteryx analytics hub,alteryx designer,amazon aws cloudformation,amazon aws glue,amazon quicksight,amazon redshift,amazon web services,anaconda enterprise,anaplan for finance,applied olap dodeca,atlassian bitbucket,atlassian jira software,azure purview,cch tagetik cpm conso,cch tagetik cpm planning,cloudera data platform,cloudera data warehouse,cloudera enterprise data hub,colibra,dax,dbt,data galaxy,databricks,databricks lakehouse platform,dataiku,denodo platform,docker,domo,fivetran,fluence,github enterprise,gitlab,gitlab ci,google bigquery,google cloud dataproc,google cloud platform,google data studio,google vertex ai,hadoop,ibm cognos analytics,ibm cognos tm1,ibm datastage,ibm db2,ibm netezza performance server,ibm spss statistics,informatica data quality (idq),informatica intelligent cloud services,informatica powercenter,java,javascript,knime analytics platform,kubernetes,looker,mdx,matillion etl,matlab,microsoft analytics platform system (ssas),microsoft azure,microsoft azure data factory,microsoft azure data lake store,microsoft azure devops services,microsoft azure machine learning,microsoft azure sql database,microsoft azure synapse analytics,microsoft fabric,microsoft power bi,microsoft sql server,microsoft sql server integration services (ssis),microsoft sql server reporting services (ssrs),microstrategy,mongodb,neo4j graph database,onestream conso,onestream planning,oracle analytics cloud,oracle business intelligence enterprise edition,oracle cloud epm,oracle data integrator (odi),oracle database,oracle financial analytics,oracle hyperion essbase,oracle hyperion financial management (hfm),oracle hyperion planning,oracle mysql,pl/sql,pentaho business analytics,postgresql,project management,prophix conso,prophix planning,python,qlik sense,qlik view,r,rstudio team,rapidminer studio,sap bw,sap bw/4hana,sap business planning and consolidation (sap bpc),sap businessobjects bi,sap crystal reports,sap data services,sap disclosure management,sap financial consolidation,sap financial information management,sap hana,sap process control,sap profitability and cost management,sas base,sas enterprise guide,sql,salesforce einstein analytics,scala,snaplogic intelligent integration platform,snowflake data cloud,spark,t-sql,tibco data science,tableau desktop and online,talend data fabric,talend open studio,talend open studio for data quality,talend platform for big data integration,teradata database,teradata vantage,terraform,vba,vena close,"vena solutions budgeting, planning & forecasting",workday adaptive planning,workiva,insightsoftware longview planning,English_level,French_level
0,2843838,Ramos,David,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0
1,2479537,Small,Carl,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,80.0,0.0,0.0,0.0,100.0,40.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,90.0,100.0
2,2533337,Evans,Carmen,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,

In [335]:
data_analyst_df = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Mapping\Output\data_analyst_scores.csv")

In [336]:
exclude_columns_1 = ['USER_ID']

In [337]:
data_analyst_df.head()

,USER_ID,weight_required_skills,weight_preferred_skills,weight_language,req_SQL_score,req_Microsoft Power BI_score,req_Databricks_score,req_DAX_score,req_Matillion ETL_score,lang_English_score,lang_French_score,pref_Cloudera Data Platform_score
0,2843838,13,1,6,150.0,0.0,0.0,0.0,0.0,300.0,300.0,0.0
1,2479537,13,1,6,0.0,0.0,0.0,0.0,0.0,270.0,300.0,0.0
2,2533337,13,1,6,150.0,0.0,0.0,40.0,0.0,150.0,300.0,0.0
3,2446382,13,1,6,240.0,240.0,0.0,0.0,0.0,180.0,300.0,0.0
4,2433124,13,1,6,0.0,0.0,0.0,0.0,0.0,300.0,300.0,0.0


In [338]:
import pandas as pd

# Assuming your DataFrame is named 'df'
columns = data_analyst_df.columns
always_list = ['USER_ID']
req_list = [col for col in columns if col.startswith('req_')] + ['weight_required_skills']
pref_list = [col for col in columns if col.startswith('pref_')] + ['weight_preferred_skills']
weights_list = ['weight_required_skills', 'weight_preferred_skills','weight_language']
lang_list = [col for col in columns if col.startswith('lang_')] + ['weight_language']

# Normalized Language Score 

In [339]:
# Step 1: Identify all language score columns (starting with 'lang_' and ending with '_score')
lang_score_columns = [col for col in data_analyst_df.columns 
                     if col.startswith('lang_') and col.endswith('_score')]

# Step 2: Calculate normalized language score
data_analyst_df['normalized_lang_score'] = (
    data_analyst_df[lang_score_columns].sum(axis=1) / 
    data_analyst_df['weight_language']
)

# Optional: Round to 2 decimal places
data_analyst_df['normalized_lang_score'] = data_analyst_df['normalized_lang_score'].round(1)


# Normalized Required score

In [340]:
# Step 1: Identify all req_*_score columns
req_score_columns = [col for col in data_analyst_df.columns 
                     if col.startswith('req_') and col.endswith('_score')]

# Step 2: Sum the req_*_scores for each row and divide by weight_required_skills
data_analyst_df['normalized_req_score'] = (
    data_analyst_df[req_score_columns].sum(axis=1) / 
    data_analyst_df['weight_required_skills']
)

# Optional: Round the result to 2 decimal places
data_analyst_df['normalized_req_score'] = data_analyst_df['normalized_req_score'].round(1)

# Normalized preferred score

In [341]:
# Step 1: Identify all pref_*_score columns
pref_score_columns = [col for col in data_analyst_df.columns 
                     if col.startswith('pref_') and col.endswith('_score')]

# Step 2: Sum the pref_*_scores for each row and divide by weight_preferred_skills
data_analyst_df['normalized_pref_score'] = (
    data_analyst_df[pref_score_columns].sum(axis=1) / 
    data_analyst_df['weight_preferred_skills']
)

# Optional: Round the result to 2 decimal places
data_analyst_df['normalized_pref_score'] = data_analyst_df['normalized_pref_score'].round(1)

# Normalized Availability Score

In [342]:
import pandas as pd

df_staffing = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Ashraf\Invert Months Processing\Input\HCK_HEC_STAFFING.csv")  

# Function to invert availability values (0 = available, 100 = unavailable)
def invert_availability_values(staffing_df: pd.DataFrame) -> pd.DataFrame:
    """
    Inverts the availability values in the month columns of the staffing DataFrame.
    Converts 0 to 100 (available) and 100 to 0 (unavailable), and inverts other values proportionally.

    Parameters:
        staffing_df (pd.DataFrame): The original staffing DataFrame with month columns.

    Returns:
        pd.DataFrame: The modified staffing DataFrame with inverted availability values.
    """
    # Define month columns (MONTH_1 to MONTH_12)
    month_cols = [f"MONTH_{i}" for i in range(1, 13)]

    # Invert values: 100 - current value for the month columns
    staffing_inverted = staffing_df.copy()
    staffing_inverted[month_cols] = 100 - staffing_inverted[month_cols]

    return staffing_inverted

# Function to save the inverted staffing DataFrame to CSV
def save_inverted_staffing_to_csv(staffing_inverted_df: pd.DataFrame, file_name: str = "staffing_inverted.csv"):
    """
    Saves the inverted staffing DataFrame to a CSV file.

    Parameters:
        staffing_inverted_df (pd.DataFrame): The modified staffing DataFrame.
        file_name (str): The name of the file to save the CSV as.
    """
    staffing_inverted_df.to_csv(file_name, index=False)
    print(f"Saved inverted staffing data to '{file_name}'.")

def add_availability_score_to_users(score_df: pd.DataFrame, selected_job: str) -> pd.DataFrame:
    import json
    import re
    import os
    os.chdir(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Outputs\Job_description_features")
    # === Step 1: Load job features JSON ===
    file_path = f"{selected_job.replace(' ', '_')}_features.json"
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            features = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"No JSON file found for job '{selected_job}'")

    # === Step 2: Extract duration from nested 'Additional Notes' ===
    notes_dict = features.get("Additional Notes", {})
    duration_text = str(notes_dict.get("Duration", ""))
    match = re.search(r"(\d+)\s*month", duration_text)
    duration = int(match.group(1)) if match else 0

    # === Step 3: Define scoring logic ===
    def check_availability_row(row, duration):
        values = row.tolist()
        max_score = 0
        for i in range(len(values) - duration + 1):
            window = values[i:i + duration]
            if all(v == 100 for v in window):
                return 100, "full block"
            elif 0 not in window:
                avg = sum(window) / duration
                max_score = max(max_score, avg)
        return max_score, "avg block" if max_score > 0 else "no block"

    # === Step 4: Score each user ===
    scores = []
    month_cols = [f"MONTH_{i}" for i in range(1, 13)]

    for user_id in score_df["USER_ID"]:
        staff_row = staffing_inverted[staffing_inverted["USER_ID"] == user_id]

        if staff_row.empty:
            scores.append(0)
            continue

        months = staff_row.iloc[0][month_cols]
        score, reason = check_availability_row(months, duration)
        scores.append(score)

    # === Step 5: Merge scores into final DataFrame ===
    result = score_df.copy()
    result["availability_score"] = scores

    return result

In [343]:
# Assuming HCK_HEC_STAFFING is already loaded
staffing_inverted = invert_availability_values(df_staffing)
score_df_Data_Engineer = add_availability_score_to_users(staffing_inverted, "Data_Analyst")

# Normalized Cosine similarity score

In [344]:
import pandas as pd

# Define the file path
file_path = r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Nehal\JSON Embedding and Matching\OUTPUTS\user_similarity_ranking.csv"

# Step 1: Read just the column names to check if 'user_rank' exists
columns_to_read = pd.read_csv(file_path, nrows=0).columns.tolist()

# Step 2: Remove 'user_rank' if it exists
if 'user_rank' in columns_to_read:
    columns_to_read.remove('user_rank')

# Step 3: Read the CSV again, excluding 'user_rank' if present
sim_df = pd.read_csv(file_path, usecols=columns_to_read)

# Convert similarity_score to percentage (0-100)
sim_df['similarity_score'] = sim_df['similarity_score'].apply(lambda x: round(x * 100,1))

# Normalized Unmatched Score

In [345]:
import pandas as pd

# Assuming your DataFrame is named 'df'
columns = data_analyst_df.columns

list1 = [col.replace('req_', '').replace('_score', '').lower() for col in columns if col.startswith('req_')]
list2 = [col.replace('pref_', '').replace('_score', '').lower() for col in columns if col.startswith('pref_')]
weights_list = ['weight_required_skills', 'weight_preferred_skills','weight_language']
lang_list = [col for col in columns if col.startswith('lang_')] + ['weight_language']

print("lang_list (cleaned lang_ columns):", lang_list)
print("list1 (req_ columns in lowercase):", list1)
print("list2 (pref_ columns in lowercase):", list2)
print("weights_list (weight columns):", weights_list)

lang_list (cleaned lang_ columns): ['lang_English_score', 'lang_French_score', 'weight_language']
list1 (req_ columns in lowercase): ['sql', 'microsoft power bi', 'databricks', 'dax', 'matillion etl']
list2 (pref_ columns in lowercase): ['cloudera data platform']
weights_list (weight columns): ['weight_required_skills', 'weight_preferred_skills', 'weight_language']


In [346]:
# Join availability_score first
data_analyst_df = pd.merge(
    data_analyst_df,
    score_df_Data_Engineer[['USER_ID', 'availability_score']],
    on='USER_ID',
    how='left'
)

# Then join similarity_score
data_analyst_df = pd.merge(
    data_analyst_df,
    sim_df[['USER_ID', 'similarity_score']],
    on='USER_ID',
    how='left'
)

# # Finally join unmatched_skills_score
# data_analyst_df = pd.merge(
#     data_analyst_df,
#     unmatched_df[['USER_ID', 'unmatched_skills_score']],
#     on='USER_ID',
#     how='left'
# )

In [347]:
# Verify the joined columns
data_analyst_df.head()

,USER_ID,weight_required_skills,weight_preferred_skills,weight_language,req_SQL_score,req_Microsoft Power BI_score,req_Databricks_score,req_DAX_score,req_Matillion ETL_score,lang_English_score,lang_French_score,pref_Cloudera Data Platform_score,normalized_lang_score,normalized_req_score,normalized_pref_score,availability_score,similarity_score
0,2843838,13,1,6,150.0,0.0,0.0,0.0,0.0,300.0,300.0,0.0,100.0,11.5,0.0,0.0,34.0
1,2479537,13,1,6,0.0,0.0,0.0,0.0,0.0,270.0,300.0,0.0,95.0,0.0,0.0,0.0,33.2
2,2533337,13,1,6,150.0,0.0,0.0,40.0,0.0,150.0,300.0,0.0,75.0,14.6,0.0,75.0,53.7
3,2446382,13,1,6,240.0,240.0,0.0,0.0,0.0,180.0,300.0,0.0,80.0,36.9,0.0,72.5,45.5
4,2433124,13,1,6,0.0,0.0,0.0,0.0,0.0,300.0,300.0,0.0,100.0,0.0,0.0,75.0,64.2


In [327]:
sim_df.head(4)

,USER_ID,similarity_score
0,2540340,100.0
1,2452725,93.7
2,2920938,89.9
3,2433117,89.5


In [326]:
score_df_Data_Engineer[['USER_ID','availability_score']].head(4)

,USER_ID,availability_score
0,2843838,0.0
1,2479537,0.0
2,2533337,75.0
3,2446382,72.5


In [247]:
unmatched_skills_cols = list(set(base_df.columns) - set(list1 + list2))
unmatched_skills_df = base_df[unmatched_skills_cols]

In [248]:
base_df.columns

Index(['USER_ID', 'LAST_NAME', 'FIRST_NAME', 'ANNEES_XP', 'airbyte',
       'alteryx analytics hub', 'alteryx designer',
       'amazon aws cloudformation', 'amazon aws glue', 'amazon quicksight',
       ...
       'teradata vantage', 'terraform', 'vba', 'vena close',
       'vena solutions budgeting, planning & forecasting',
       'workday adaptive planning', 'workiva',
       'insightsoftware longview planning', 'English_level', 'French_level'],
      dtype='object', length=139)

In [249]:
unmatched_skills_cols

['amazon quicksight',
 'oracle data integrator (odi)',
 'microsoft analytics platform system (ssas)',
 'azure purview',
 'matlab',
 'vba',
 'alteryx designer',
 'insightsoftware longview planning',
 'sas enterprise guide',
 'colibra',
 'rstudio team',
 'microsoft fabric',
 'sap crystal reports',
 'sap business planning and consolidation (sap bpc)',
 'project management',
 'mdx',
 'onestream conso',
 'spark',
 'microsoft azure synapse analytics',
 'google cloud dataproc',
 'talend open studio',
 'sap financial information management',
 'alteryx analytics hub',
 'sap bw',
 'tibco data science',
 'snowflake data cloud',
 'google bigquery',
 'javascript',
 'amazon web services',
 'ibm cognos tm1',
 'prophix planning',
 'terraform',
 'microstrategy',
 'hadoop',
 'cch tagetik cpm conso',
 'neo4j graph database',
 'microsoft sql server integration services (ssis)',
 'fivetran',
 'amazon aws glue',
 'oracle financial analytics',
 'cloudera enterprise data hub',
 'ibm cognos analytics',
 'micro

In [250]:
unmatched_skills_df

,amazon quicksight,oracle data integrator (odi),microsoft analytics platform system (ssas),azure purview,matlab,vba,alteryx designer,insightsoftware longview planning,sas enterprise guide,colibra,rstudio team,microsoft fabric,sap crystal reports,sap business planning and consolidation (sap bpc),project management,mdx,onestream conso,spark,microsoft azure synapse analytics,google cloud dataproc,talend open studio,sap financial information management,alteryx analytics hub,sap bw,tibco data science,snowflake data cloud,google bigquery,javascript,amazon web services,ibm cognos tm1,prophix planning,terraform,microstrategy,hadoop,cch tagetik cpm conso,neo4j graph database,microsoft sql server integration services (ssis),fivetran,amazon aws glue,oracle financial analytics,cloudera enterprise data hub,ibm cognos analytics,microsoft sql server,cloudera data warehouse,domo,microsoft sql server reporting services (ssrs),ibm spss statistics,microsoft azure data factory,atlassian jira software,anaconda enterprise,sap data services,oracle hyperion financial management (hfm),microsoft azure,prophix conso,kubernetes,oracle cloud epm,USER_ID,salesforce einstein analytics,qlik sense,fluence,mongodb,airbyte,r,sap disclosure management,oracle business intelligence enterprise edition,sas base,applied olap dodeca,rapidminer studio,workday adaptive planning,"vena solutions budgeting, planning & forecasting",teradata database,amazon redshift,denodo platform,github enterprise,LAST_NAME,sap profitability and cost management,informatica data quality (idq),oracle hyperion planning,docker,anaplan for finance,oracle mysql,cch tagetik cpm planning,gitlab,FIRST_NAME,t-sql,English_level,postgresql,microsoft azure devops services,microsoft azure data lake store,python,looker,talend open studio for data quality,microsoft azure sql database,onestream planning,data galaxy,informatica powercenter,talend platform for big data integration,amazon aws cloudformation,oracle analytics cloud,dbt,knime analytics platform,microsoft azure machine learning,qlik view,snaplogic intelligent integration platform,scala,ANNEES_XP,google data studio,teradata vantage,workiva,atlassian bitbucket,databricks lakehouse platform,sap bw/4hana,informatica intelligent cloud services,sap financial consolidation,vena close,talend data fabric,French_level,google vertex ai,ibm netezza performance server,oracle hyperion essbase,pl/sql,tableau desktop and online,gitlab ci,oracle database,ibm db2,pentaho business analytics,sap process control,sap businessobjects bi,ibm datastage,dataiku,java,google cloud platform,sap hana
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2843838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Ramos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,David,0.0,100.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,80.0,2479537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Small,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,Carl,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0
2,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2533337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [170]:
skills_df = pd.read_csv(r"C:\Users\hp\Documents\GitHub\HEC_DS_Challenge\Datasets\Bronze\HCK_HEC_SKILLS.csv")

In [173]:
skills_df['DOMAIN_DSC'].unique()

array(['Data Platform', 'Data Science', 'Data Governance',
       'Data Analytics', 'Agile Delivery', 'Data Management',
       'Financial Close', 'Financial Planning and Analysis',
       'Programming Language', 'Data Integration'], dtype=object)

In [175]:
skills_df[skills_df['DOMAIN_DSC'] == 'Agile Delivery']

,USER_ID,SKILLS_DSC,DOMAIN_DSC,LEVEL_VAL
701,2836571,Atlassian JIRA Software,Agile Delivery,50.0
702,2836571,Microsoft Azure DevOps Services,Agile Delivery,50.0
703,2836571,Atlassian JIRA Software,Agile Delivery,50.0
704,2836571,Microsoft Azure DevOps Services,Agile Delivery,50.0
705,2836571,Atlassian JIRA Software,Agile Delivery,50.0
...,...,...,...,...
3399,2977311,Atlassian JIRA Software,Agile Delivery,NaN
3400,2977311,Project Management,Agile Delivery,NaN
3401,2977311,Atlassian JIRA Software,Agile Delivery,NaN
3402,2903451,Project Management,Agile Delivery,NaN


In [ ]:
candidate_technical_scores_df['USER_ID'] 

,USER_ID,LAST_NAME,french_score,english_score,experience_score,availability_score
0,2843838,Ramos,100.0,100.0,30,100
1,2479537,Small,100.0,90.0,50,100
2,2533337,Evans,100.0,50.0,50,100
3,2446382,Pittman,100.0,60.0,50,100
4,2433124,Thomas,100.0,100.0,50,100
...,...,...,...,...,...,...
114,2713733,Williams,80.0,100.0,50,100
115,2456947,Riddle,100.0,100.0,50,100
116,2447672,Gonzalez,100.0,80.0,50,100
117,2789547,Williams,100.0,70.0,30,100


In [ ]:
skills_df[skills_df['USER_ID'] == 2843838]

,USER_ID,SKILLS_DSC,DOMAIN_DSC,LEVEL_VAL
525,2479537,Oracle Analytics Cloud,Data Analytics,60.0
1600,2479537,Oracle Hyperion Financial Management (HFM),Financial Close,40.0
1676,2479537,Oracle Hyperion Planning,Financial Planning and Analysis,100.0
1677,2479537,Applied OLAP Dodeca,Financial Planning and Analysis,80.0
1678,2479537,Oracle Cloud EPM,Financial Planning and Analysis,80.0
1679,2479537,Oracle Hyperion Essbase,Financial Planning and Analysis,100.0
2023,2479537,Java,Programming Language,60.0
2024,2479537,VBA,Programming Language,100.0


In [171]:
skills_df['SKILLS_DSC'].unique()

array(['Snowflake Data Cloud', 'Microsoft Azure', 'Amazon Web Services',
       'Google Cloud Platform', 'Databricks Lakehouse Platform',
       'Microsoft Fabric', 'KNIME Analytics Platform', 'SAS Base',
       'Databricks', 'RStudio Team', 'Dataiku', 'IBM SPSS Statistics',
       'Microsoft Azure Machine Learning', 'Anaconda Enterprise',
       'Matlab', 'Google Vertex AI', 'TIBCO Data Science',
       'RapidMiner Studio', 'Data Galaxy', 'Azure Purview',
       'Tableau Desktop and Online', 'Microsoft Power BI',
       'Amazon QuickSight', 'Google Data Studio', 'Looker',
       'Microstrategy', 'Qlik View',
       'Microsoft SQL Server Reporting Services (SSRS)',
       'Salesforce Einstein Analytics', 'Qlik Sense',
       'Oracle Business Intelligence Enterprise Edition',
       'IBM Cognos Analytics', 'Alteryx Analytics Hub',
       'SAP BusinessObjects BI', 'Oracle Analytics Cloud',
       'SAS Enterprise Guide', 'SAP Crystal Reports',
       'Pentaho Business Analytics', 'Domo', 

In [180]:
skills_df[skills_df['DOMAIN_DSC'] == 'Agile Delivery']

,USER_ID,SKILLS_DSC,DOMAIN_DSC,LEVEL_VAL
701,2836571,Atlassian JIRA Software,Agile Delivery,50.0
702,2836571,Microsoft Azure DevOps Services,Agile Delivery,50.0
703,2836571,Atlassian JIRA Software,Agile Delivery,50.0
704,2836571,Microsoft Azure DevOps Services,Agile Delivery,50.0
705,2836571,Atlassian JIRA Software,Agile Delivery,50.0
...,...,...,...,...
3399,2977311,Atlassian JIRA Software,Agile Delivery,NaN
3400,2977311,Project Management,Agile Delivery,NaN
3401,2977311,Atlassian JIRA Software,Agile Delivery,NaN
3402,2903451,Project Management,Agile Delivery,NaN


In [190]:
skills_df['DOMAIN_DSC'].unique()

array(['Data Platform', 'Data Science', 'Data Governance',
       'Data Analytics', 'Agile Delivery', 'Data Management',
       'Financial Close', 'Financial Planning and Analysis',
       'Programming Language', 'Data Integration'], dtype=object)

In [188]:
from fuzzywuzzy import process

def match_skills(list1: list, list2: list, threshold: int = 85) -> dict:
    """
    Fuzzy matches skills between two lists and returns matches with scores ≥ threshold.
    
    Args:
        list1 (list): Source skills to match FROM (e.g., user skills)
        list2 (list): Target skills to match TO (e.g., required skills)
        threshold (int): Minimum match score (0-100)
    
    Returns:
        dict: { "source_skill": ("matched_skill", score) }
    """
    matches = {}
    for skill in list1:
        match, score = process.extractOne(skill, list2)
        if score >= threshold:
            matches[skill] = (match, score)
    return matches

In [ ]:
required_skills = skills_df[skills_df['DOMAIN_DSC'] == 'Agile Delivery']['SKILLS_DSC'].unique()


In [189]:
matches = match_skills(all_skills, required_skills)
print(matches)

{'Jira': ('Atlassian JIRA Software', 90), 'Microsoft Teams': ('Microsoft Azure DevOps Services', 86), 'Jira Dashboards': ('Atlassian JIRA Software', 86)}


In [186]:
# Extract all skills from the JSON file
def extract_all_skills(job_data):
    required_skills = [skill['skill'] for skill in job_data.get('Required Skills', [])]
    preferred_skills = [skill['skill'] for skill in job_data.get('Preferred Skills', [])]
    all_skills = required_skills + preferred_skills
    return all_skills

# Get all skills from job_requirements_data
all_skills = extract_all_skills(job_requirements_data)
print("All Skills:", all_skills)

All Skills: ['Scrum', 'Kanban', 'Lean', 'Agile Methodologies', 'Jira', 'Confluence', 'Microsoft Teams', 'Slack', 'Miro', 'Jira Dashboards', 'Power BI', 'Advanced Excel', 'SAFe']


In [179]:
job_requirements_data

{'Job Title': 'Scrum Master',
 'Required Skills': [{'skill': 'Scrum', 'level': 3},
  {'skill': 'Kanban', 'level': 2},
  {'skill': 'Lean', 'level': 2},
  {'skill': 'Agile Methodologies', 'level': 2},
  {'skill': 'Jira', 'level': 2},
  {'skill': 'Confluence', 'level': 2},
  {'skill': 'Microsoft Teams', 'level': 2},
  {'skill': 'Slack', 'level': 2},
  {'skill': 'Miro', 'level': 2},
  {'skill': 'Jira Dashboards', 'level': 2},
  {'skill': 'Power BI', 'level': 2},
  {'skill': 'Advanced Excel', 'level': 2}],
 'Preferred Skills': [{'skill': 'SAFe', 'level': 1}],
 'Experience Required': 3.0,
 'Languages': [{'language': 'French', 'level': 3},
  {'language': 'English', 'level': 2}],
 'Responsibilities': ['Establish and maintain an effective agile framework for collective performance.',
  'Facilitate all scrum ceremonies (daily scrum, sprint planning, sprint review, retrospective, refinement).',
  'Proactively manage obstacles to ensure smooth delivery.',
  'Support the team in developing autonomy